In [16]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("tripdata.dat")
database = db.Database("tripdata",pandas)

# The Pandas data structure is available as database.data. Use all the
# Pandas functions to invesigate the database
#print(database.data.describe())

from headers import *

Below are all of the variables declarations:

In [17]:
# Alternative specific constants
ASC_AUTO_DRIVE = Beta('ASC_AUTO_DRIVE',0,None,None,0)
ASC_AUTO_PASS = Beta('ASC_AUTO_PASS',0,None,None,0)
ASC_METRO = Beta('ASC_METRO',0,None,None,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None,None,0)
ASC_WALK = Beta('ASC_WALK',0,None,None,1)

# Sociodemo variables
B_SEX = Beta('B_SEX',0,None,None,0) # 1 if female; 0 if otherwise
B_AGE = Beta('B_AGE',0,None,None,0) # age
B_EMP = Beta('B_EMP',0,None,None,0) # parttime or fulltime
B_WKWK = Beta('B_WKWK',0,None,None,0) # work week
B_INC = Beta('B_INC',0,None,None,0) # income
B_NVEH = Beta('B_NVEH',0,None,None,0) # number of household vehicles

# Generalized travel variables
B_IVTT = Beta('B_IVTT',0,None,None,0) # in vehicle travel time
B_DIST = Beta('B_DIST',0,None,None,0) # distance

# Car-related variables
B_DLIC = Beta('B_DLIC',0,None,None,0) # drivers license 1 or 0

# Train-related variables
B_NTERM = Beta('B_NTERM',0,None,None,0) # commuter train access station
B_GFARE = Beta('B_GFARE',0,None,None,0) # commuter train fare
B_GWAIT = Beta('B_GWAIT',0,None,None,0) # commuter train wait time
B_GACCA = Beta('B_GACCA',0,None,None,0) # commuter train wait time
B_EGRESS = Beta('B_GACCA',0,None,None,0) # commuter train wait time


# Metro-related variables
B_MSUB = Beta('B_MSUB',0,None,None,0) # closest metro station to destination
B_MDIS = Beta('B_MDIS',0,None,None,0) # distance from metro to destination
B_TFARE = Beta('B_TFARE',0,None,None,0) # transit fare
B_TWAIT = Beta('B_TWAIT',0,None,None,0) # transit wait time

The next cell defines new variables in order to exclude crazy values

In [18]:
# Updating variables to account for 'missing' values 
nveh_fix =  DefineVariable('nveh_fix', nveh * (nveh != 9), database)
gfare_fix = DefineVariable('gfare_fix', gfare * (gfare != 9999), database)
tfare_fix = DefineVariable('tfare_fix', tfare * (tfare != 9999), database)
gwait_fix = DefineVariable('gwait_fix', gwait * (gwait != 9999), database)
twait_fix = DefineVariable('twait_fix', twait * (twait != 9999), database)

In [19]:
V1 = ASC_AUTO_DRIVE + \
     B_DIST * trdist + \
     B_DLIC * dlic + \
     B_IVTT * ivtt1 + \
     B_SEX * female
V2 = ASC_AUTO_PASS + \
     B_NVEH * nveh_fix + \
     B_IVTT * ivtt2 + \
     B_SEX * female
V3 = ASC_METRO + \
     B_TFARE * tfare_fix + \
     B_IVTT * ivtt3 + \
     B_SEX * female
V4 = ASC_TRAIN + \
     B_NTERM * nterm + \
     B_GFARE * gfare_fix + \
     B_IVTT * ivtt4 + \
     B_SEX * female
V5 = ASC_WALK + \
     B_AGE * age * age2 + \
     B_IVTT * ivtt5 + \
     B_SEX * female

In [20]:
# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3,
     4: V4,
     5: V5}

# Associate the availability conditions with the alternatives

av = {1: avail1,
      2: avail2,
      3: avail3,
      4: avail4,
      5: avail5}

logprob = bioLogLogit(V,av,choice)
biogeme  = bio.BIOGEME(database,logprob) # This is my log likelihood function
biogeme.modelName = "MNL_Model_final"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_AUTO_DRIVE=	-158
ASC_AUTO_PASS=	-14.7
ASC_METRO=	-12.1
ASC_TRAIN=	-9.54
B_AGE=	26.1
B_DIST=	-0.0328
B_DLIC=	144
B_GFARE=	-1.36
B_IVTT=	-0.0328
B_NTERM=	-0.0768
B_NVEH=	-0.386
B_SEX=	0
B_TFARE=	-1.04
                     Value       Std err        t-test       p-value  \
ASC_AUTO_DRIVE -157.654225  3.426298e+02 -4.601300e-01  6.454229e-01   
ASC_AUTO_PASS   -14.697491  6.852596e+02 -2.144806e-02  9.828882e-01   
ASC_METRO       -12.148864  6.852597e+02 -1.772885e-02  9.858552e-01   
ASC_TRAIN        -9.535084  6.852602e+02 -1.391454e-02  9.888982e-01   
B_AGE            26.066287  2.769343e-06  9.412445e+06  0.000000e+00   
B_DIST           -0.032847  1.126415e-02 -2.916054e+00  3.544896e-03   
B_DLIC          144.421483  3.426298e+02  4.215088e-01  6.733836e-01   
B_GFARE          -1.364044  1.971471e-01 -6.918916e+00  4.551026e-12   
B_IVTT           -0.032785  1.230498e-02 -2.664400e+00  7.712584e-03   
B_NTERM          -0.076760  3.572613e-02 -2.148581e+00  3.166759e-02   
B_NVE